# Ensembles & Pipelines
Finally you'll learn how to make your models more efficient. You'll find out how to use pipelines to make your code clearer and easier to maintain. Then you'll use cross-validation to better test your models and select good model parameters. Finally you'll dabble in two types of ensemble model.

## Flight duration model: Pipeline stages
You're going to create the stages for the flights duration model pipeline. You will use these in the next exercise to build a pipeline and to create a regression model.|

In [2]:
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler
from pyspark.sql.functions import round, col
from pyspark.ml.regression import LinearRegression

# File Path
file_path = ".../data/datacamp/"

# Load in data
flights = spark.read.parquet(file_path + "flights_ml") \
    .drop('flight') \
    .filter('delay IS NOT NULL') \
    .dropna() \
    .withColumn('km', round(col("mile") * 1.60934, 0)) \
    .drop('mile')

# Convert categorical strings to index values
indexer = StringIndexer(inputCol='org', outputCol='org_idx')

# One-hot encode index values
onehot = OneHotEncoderEstimator(
    inputCols=['org_idx', 'dow'],
    outputCols=['org_dummy', 'dow_dummy']
)

# Assemble predictors into a single column
assembler = VectorAssembler(inputCols=['km', 'org_dummy', 'dow_dummy'], outputCol='features')

# A linear regression object
regression = LinearRegression(labelCol='duration')

## Flight duration model: Pipeline model
You're now ready to put those stages together in a pipeline.

You'll construct the pipeline and then train the pipeline on the training data. This will apply each of the individual stages in the pipeline to the training data in turn. None of the stages will be exposed to the testing data at all: there will be no leakage!

Once the entire pipeline has been trained it will then be used to make predictions on the testing data.

In [3]:
# Import class for creating a pipeline
from pyspark.ml import Pipeline

# Split into train and test data
flights_train, flights_test = flights.randomSplit([0.8, 0.2], seed=17) 

# Construct a pipeline
pipeline = Pipeline(stages=[indexer, onehot, assembler, regression])

# Train the pipeline on the training data
pipeline = pipeline.fit(flights_train)

# Make predictions on the testing data
predictions = pipeline.transform(flights_test)

## SMS spam pipeline
You haven't looked at the SMS data for quite a while. Last time we did the following:

- split the text into tokens
- removed stop words
- applied the hashing trick
- converted the data from counts to IDF and
- trained a linear regression model.

Each of these steps was done independently. This seems like a great application for a pipeline!

In [4]:
from pyspark.sql.types import StringType, StructField, StructType
from pyspark.sql.functions import lit, regexp_replace
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF
from pyspark.ml.classification import LogisticRegression


# Load data
sms_train, sms_test = spark.read \
    .schema(StructType([StructField('text', StringType(), True)])) \
    .csv(file_path + "spam") \
    .withColumn('label', lit(1)) \
    .union(
spark.read \
    .schema(StructType([StructField('text', StringType(), True)])) \
    .csv(file_path + "no_spam") \
    .withColumn('label', lit(0))
) \
    .withColumn('text', regexp_replace(col("text"), '[_():;,.!?\\-]', ' ')) \
    .withColumn('text', regexp_replace(col("text"), '[0-9]', ' ')) \
    .withColumn('text', regexp_replace(col("text"), ' +', ' ')) \
    .repartition(300) \
    .randomSplit([0.8, 0.2], seed=17)

# Break text into tokens at non-word characters
tokenizer = Tokenizer(inputCol='text', outputCol='words')

# Remove stop words
remover = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol='terms')

# Apply the hashing trick and transform to TF-IDF
hasher = HashingTF(inputCol=remover.getOutputCol(), outputCol="hash")
idf = IDF(inputCol=hasher.getOutputCol(), outputCol="features")

# Create a logistic regression object and add everything to a pipeline
logistic = LogisticRegression()
pipeline = Pipeline(stages=[tokenizer, remover, hasher, idf, logistic])

# Train the pipeline on the training data
pipeline = pipeline.fit(sms_train)

# Make predictions on the testing data
predictions = pipeline.transform(sms_test)

predictions.show(5)

+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|                text|label|               words|               terms|                hash|            features|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|           My friend|    0|        [my, friend]|            [friend]|(262144,[158870],...|(262144,[158870],...|[-15.789917780714...|[1.38843571898580...|       1.0|
|Mmmmmmm *snuggles...|    0|[mmmmmmm, *snuggl...|[mmmmmmm, *snuggl...|(262144,[4481,354...|(262144,[4481,354...|[16.1186062426889...|[0.99999990005105...|       0.0|
|I'm not sure if i...|    0|[i'm, not, sure, ...|[sure, still, ava...|(262144,[36200,13...|(262144,[36200,13...|[29.5416148181337...|[0.99999999999985...|       0.0|
|Con

## Cross validating simple flight duration model
You've already built a few models for predicting flight duration and evaluated them with a simple train/test split. However, cross-validation provides a much better way to evaluate model performance.

In this exercise you're going to train a simple model for flight duration using cross-validation. Travel time is usually strongly correlated with distance, so using the km column alone should give a decent model.

The data have been randomly split into flights_train and flights_test.

In [5]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

# Create an assembler object and split into train and test data
flights_train, flights_test = VectorAssembler(inputCols=['km'], outputCol='features') \
    .transform(flights) \
    .select('features', 'duration') \
    .randomSplit([0.8, 0.2], seed=17)

# Create an empty parameter grid
params = ParamGridBuilder().build()

# Create objects for building and evaluating a regression model
regression = LinearRegression(labelCol='duration')
evaluator = RegressionEvaluator(labelCol='duration')

# Create a cross validator
cv = CrossValidator(estimator=regression, estimatorParamMaps=params, evaluator=evaluator, numFolds=5)

# Train and test model on multiple folds of the training data
cv = cv.fit(flights_train)

# NOTE: Since cross-valdiation builds multiple models, the fit() method can take a little while to complete.

## Cross validating flight duration model pipeline
The cross-validated model that you just built was simple, using km alone to predict duration.

Another important predictor of flight duration is the origin airport. Flights generally take longer to get into the air from busy airports. Let's see if adding this predictor improves the model!

In this exercise you'll add the org field to the model. However, since org is categorical, there's more work to be done before it can be included: it must first be transformed to an index and then one-hot encoded before being assembled with km and used to build the regression model. We'll wrap these operations up in a pipeline.

The following objects have already been created:

- params — an empty parameter grid
- evaluator — a regression evaluator
- regression — a LinearRegression object with labelCol='duration'.

In [6]:
# Create an indexer for the org field
indexer = StringIndexer(inputCol='org', outputCol='org_idx')

# Create an one-hot encoder for the indexed org field
onehot = OneHotEncoderEstimator(inputCols=[indexer.getOutputCol()], outputCols=['org_dummy'])

# Assemble the km and one-hot encoded fields
assembler = VectorAssembler(inputCols=['km', 'org_dummy'], outputCol='features')

# Create a pipeline and cross-validator.
pipeline = Pipeline(stages=[indexer, onehot, assembler, regression])
cv = CrossValidator(estimator=pipeline,
          estimatorParamMaps=params,
          evaluator=evaluator)

## Optimizing flights linear regression
Up until now you've been using the default hyper-parameters when building your models. In this exercise you'll use cross validation to choose an optimal (or close to optimal) set of model hyper-parameters.

The following have already been created:

- regression — a LinearRegression object
- pipeline — a pipeline with string indexer, one-hot encoder, vector assembler and linear regression and
- evaluator — a RegressionEvaluator object.

In [7]:
# Create parameter grid
params = ParamGridBuilder()

# Add grids for two parameters
params = params.addGrid(regression.regParam, [0.01, 0.1, 1.0, 10.0]) \
               .addGrid(regression.elasticNetParam, [0.0, 0.5, 1.0])

# Build the parameter grid
params = params.build()
print('Number of models to be tested: ', len(params))

# Create cross-validator
cv = CrossValidator(estimator=pipeline, estimatorParamMaps=params, evaluator=evaluator, numFolds=5)

Number of models to be tested:  12


## Dissecting the best flight duration model
You just set up a CrossValidator to find good parameters for the linear regression model predicting flight duration.

Now you're going to take a closer look at the resulting model, split out the stages and use it to make predictions on the testing data.

The following have already been created:

cv — a trained CrossValidatorModel object and
evaluator — a RegressionEvaluator object.

In [8]:
# Create an assembler object and split into train and test data
flights_train, flights_test = flights.randomSplit([0.8, 0.2], seed=17)

# Train model
cv = cv.fit(flights_train)

# Get the best model from cross validation
best_model = cv.bestModel

# Look at the stages in the best model
print(best_model.stages)

# Get the parameters for the LinearRegression object in the best model
best_model.stages[3].extractParamMap()

# Generate predictions on testing data using the best model then calculate RMSE
predictions = best_model.transform(flights_test)
evaluator.evaluate(predictions)

[StringIndexer_e0f98f45a085, OneHotEncoderEstimator_45160363425f, VectorAssembler_de9054019135, LinearRegression_85a330a851ed]


11.032234663128339

## SMS spam optimised
The pipeline you built earlier for the SMS spam model used the default parameters for all of the elements in the pipeline. It's very unlikely that these parameters will give a particularly good model though.

In this exercise you'll set up a parameter grid which can be used with cross validation to choose a good set of parameters for the SMS spam classifier.

The following are already defined:

hasher — a HashingTF object and
logistic — a LogisticRegression object.

In [9]:
# Create parameter grid
params = ParamGridBuilder()

# Add grid for hashing trick parameters
params = params.addGrid(hasher.numFeatures, [1024, 4096, 16384]) \
               .addGrid(hasher.binary, [True, False])

# Add grid for logistic regression parameters
params = params.addGrid(logistic.regParam, [0.01, 0.1, 1.0, 10]) \
               .addGrid(logistic.elasticNetParam, [ 0.0, 0.5, 1.0 ])

# Build parameter grid
params = params.build()

## Delayed flights with Gradient-Boosted Trees
You've previously built a classifier for flights likely to be delayed using a Decision Tree. In this exercise you'll compare a Decision Tree model to a Gradient-Boosted Trees model.

In [10]:
# Import the classes required
from pyspark.ml.classification import DecisionTreeClassifier, GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Create 'label' column indicating whether flight delayed (1) or not (0)
flights = flights.withColumn('label', (flights.delay >= 15).cast('integer'))

# Create an assembler object and split into train and test data
flights_train, flights_test = VectorAssembler(inputCols=['mon', 'depart', 'duration'], outputCol='features') \
    .transform(flights) \
    .select('features', 'label') \
    .randomSplit([0.8, 0.2], seed=17)


# Create model objects and train on training data
tree = DecisionTreeClassifier().fit(flights_train)
gbt = GBTClassifier().fit(flights_train)

# Compare AUC on testing data
evaluator = BinaryClassificationEvaluator()
evaluator.evaluate(tree.transform(flights_test))
evaluator.evaluate(gbt.transform(flights_test))

# Find the number of trees and the relative importance of features
print(gbt.getNumTrees)
print(gbt.featureImportances)

20
(3,[0,1,2],[0.34256163370799675,0.26892574409830017,0.3885126221937031])


## Delayed flights with a Random Forest
In this exercise you'll bring together cross validation and ensemble methods. You'll be training a Random Forest classifier to predict delayed flights, using cross validation to choose the best values for model parameters.

You'll find good values for the following parameters:

- featureSubsetStrategy — the number of features to consider for splitting at each node and
- maxDepth — the maximum number of splits along any branch.
- Unfortunately building this model takes too long, so we won't be running the .fit() method on the pipeline.

In [16]:
from pyspark.ml.classification import RandomForestClassifier

# Create a random forest classifier
forest = RandomForestClassifier()

# Create a parameter grid
params = ParamGridBuilder() \
            .addGrid(forest.featureSubsetStrategy, ['all', 'onethird', 'sqrt', 'log2']) \
            .addGrid(forest.maxDepth, [2, 5, 10]) \
            .build()

# Create a binary classification evaluator
evaluator = BinaryClassificationEvaluator()

# Create a cross-validator
cv = CrossValidator(estimator=forest, estimatorParamMaps=params, evaluator=evaluator, numFolds=2)

## Evaluating Random Forest
In this final exercise you'll be evaluating the results of cross-validation on a Random Forest model.

The following have already been created:

- cv - a cross-validator which has already been fit to the training data
- evaluator — a BinaryClassificationEvaluator object and
- flights_test — the testing data.

In [17]:
# Train Model
cv = cv.fit(flights_train)

# Average AUC for each parameter combination in grid
avg_auc = cv.avgMetrics

# Average AUC for the best model
best_model_auc = max(cv.avgMetrics)

# What's the optimal parameter value?
opt_max_depth = cv.bestModel.explainParam('maxDepth')
opt_feat_substrat = cv.bestModel.explainParam('featureSubsetStrategy')

# AUC for best model on testing data
best_auc = evaluator.evaluate(cv.transform(flights_test))

In [18]:
best_auc

0.6829244527549139